# A Guided Tour of Ray Core: Remote Objects

[*Remote Objects*](https://docs.ray.io/en/latest/walkthrough.html#objects-in-ray)
implement a [*shared-memory object store*](https://en.wikipedia.org/wiki/Shared_memory) pattern.

Objects are immutable, and can be accessed from anywhere on the cluster.

---

First, let's start Ray…

In [ ]:
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

## Remote Objects example

To start, we'll define a remote object...

In [ ]:
%%time

num_list = [ 23, 42, 93 ]

obj_ref = ray.put(num_list)
obj_ref

Then retrieve the value of this object reference

In [ ]:
%%time

ray.get(obj_ref)

Let's combine use of a remote function with a remote object, to illustrate *composable futures*:

In [ ]:
@ray.remote
def my_function (num_list):
    return sum(num_list)

In other words, the remote function `myfunction()` will sum the list of integers in the remote object `num_list`:

In [ ]:
%%time

calc_ref = my_function.remote(obj_ref)

In [ ]:
%%time

ray.get(calc_ref)

You can gather the values of multiple object references in parallel using collections:

In [ ]:
%%time

ray.get([ray.put(i) for i in range(3)])

Now let's set a timeout to return early from attempted access of a remote object that is blocking for too long...

In [ ]:
import time

@ray.remote
def long_running_function ():
    time.sleep(10)
    return 42

In [ ]:
%%time

from ray.exceptions import GetTimeoutError

obj_ref = long_running_function.remote()

try:
    ray.get(obj_ref, timeout=6)
except GetTimeoutError:
    print("`get` timed out")

Then shutdown Ray

In [ ]:
ray.shutdown()